<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/NN_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by Neural Network and TfIdfVectorizer**

----------

 import library

  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

**1- Import Data set**

In [2]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")
df.head()

,text,targe
0,أكد وزير الاتصال الناطق الرسمي باسم الحكومة مص...,0
1,عبدالاله بوسحابة أخبارنا المغربية أثار فيلم أل...,0
2,عبدالاله بوسحابة اخبارنا المغربية أبدى القائمو...,0
3,أخبارنا المغربية اختار الفنان المغربي البشير ع...,0
4,المادة الأولى على ملك غرناطة القادة الفقهاء ال...,0


------
shuffle  
بعثرة البيانات

In [3]:
df=shuffle(df)
df

,text,targe
17956,سلط وزير الثقافة والاتصال محمد الأعرج بمقر منظ...,0
21666,وضعت المصالح الأمنية بفاس الأحد الماضي يدها عل...,1
929,أخبارنا المغربية الوزاني استضافت قناة فرانس يو...,0
54034,يصدرعن دار الوطن للصحافة والطباعة والنشر بالرب...,0
19059,بعد النجاح الذي حققته سلسلة لوبيرج شرع الممثل ...,0
...,...,...
65351,ادعى الساحر الغاني نانا كواكو بونسام أنه السبب...,4
11606,أخبارنا المغربية اعتبر رئيس حزب التجمع الوطني ...,3
59191,أعلنت شركة أديداس الألمانية للملابس والأحذية و...,2
58439,سجلت حركة النقل الجوي التي تربط مختلف مطارات ا...,2


------
**2- Clean Data**

تنظيف البيانات

In [4]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
17956  سلط وزير الثقافه والاتصال محمد الاعرج بمقر منظ...      0
21666  وضعت المصالح الامنيه بفاس الاحد الماضي يدها عل...      1
929    اخبارنا المغربيه الوزاني استضافت قناه فرانس يو...      0
54034  يصدرعن دار الوطن لصحافه والطباعه والنشر بالربا...      0
19059  بعد النجاح الذي حقته سلسله لوبيرج شرع المثل وا...      0
...                                                  ...    ...
65351  ادعي الساحر الغاني نانا كواكو بونسام انه السب ...      4
11606  اخبارنا المغربيه اعتبر رئيس حزب التجمع الوطني ...      3
59191  اعلنت شركه اديداس الالمانيه لملابس والاحذيه وا...      2
58439  سجلت حركه النقل الجوي التي تربط مختلف مطارات ا...      2
1236   اخبارنا المغربيه سناء الوردي قر والد الفنانه ا...      0

[68312 rows x 2 columns]


---------
accuracy calculation function

عمل تابع لحساب الدقة

In [5]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:** TfIdtVectorizer

**4- classification:** Neural Network

Neural Network & TfIdtVectorizer

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [6]:
%%time
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras import layers
from keras.layers import Dense
from keras.backend import clear_session
from keras.utils.np_utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer

x = clean_df['text'].values
y = clean_df['targe'].values

tfidf = TfidfVectorizer(max_features=5000)
strtfdKFold = StratifiedKFold(n_splits=4)
kfold = strtfdKFold.split(x, y)
result_NNClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    tfidf.fit(X_train)
    X_train = tfidf.transform(X_train).toarray()
    X_test  = tfidf.transform(X_test).toarray()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    input_dim = X_train.shape[1]

    model = Sequential()
    model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(5, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    model.fit(X_train, y_train,epochs=2,verbose=True,batch_size=10)#validation_split=0.1,

    y_pred=(model.predict(X_test))
    y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
    y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]

    clear_session()
    result_NNClassifier.append(calculate_results(y_classes_test, y_classes_pre))
result_NNClassifier

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50010     
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
Total params: 50,065
Trainable params: 50,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
5124/5124 [==============================] - 14s 3ms/step - loss: 0.1287 - accuracy: 0.9194
Epoch 2/2
5124/5124 [==============================] - 11s 2ms/step - loss: 0.0564 - accuracy: 0.9541
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50010     
                          

In [7]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_NNClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_NNClassifier)
f1=f1/len(result_NNClassifier)
precision=precision/len(result_NNClassifier)
recall=recall/len(result_NNClassifier)
# (result_DecisionTreeClassifier)
results_NNClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_NNClassifier

[{'accuracy': 94.72127883827146,
  'f1': 0.9471813461397798,
  'precision': 0.9472089449863148,
  'recall': 0.9472127883827147}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [8]:
print('NN_Classifier results in dataset 1:')
results_NNClassifier

NN_Classifier results in dataset 1:


[{'accuracy': 96.41724708252787,
  'f1': 0.9642110793616473,
  'precision': 0.9643367428610339,
  'recall': 0.9641724708252787}]

In [8]:
print('NN_Classifier results in dataset 2:')
results_NNClassifier

NN_Classifier results in dataset 2:


[{'accuracy': 96.31393603509989,
  'f1': 0.9631651460998529,
  'precision': 0.9632074203556251,
  'recall': 0.9631393603509988}]

In [8]:
print('NN_Classifier results in dataset 3:')
results_NNClassifier

NN_Classifier results in dataset 3:


[{'accuracy': 95.05879502111596,
  'f1': 0.9505681788205832,
  'precision': 0.9506680801119126,
  'recall': 0.9505879502111595}]

In [8]:
print('NN_Classifier results in dataset 4:')
results_NNClassifier

NN_Classifier results in dataset 4:


[{'accuracy': 94.72127883827146,
  'f1': 0.9471813461397798,
  'precision': 0.9472089449863148,
  'recall': 0.9472127883827147}]

In [9]:
Final_Result=[
  {'accuracy': 96.41724708252787,
  'f1': 0.9642110793616473,
  'precision': 0.9643367428610339,
  'recall': 0.9641724708252787}
  ,
  {'accuracy': 96.31393603509989,
  'f1': 0.9631651460998529,
  'precision': 0.9632074203556251,
  'recall': 0.9631393603509988}
  ,
  {'accuracy': 95.05879502111596,
  'f1': 0.9505681788205832,
  'precision': 0.9506680801119126,
  'recall': 0.9505879502111595}
  ,
  {'accuracy': 94.72127883827146,
  'f1': 0.9471813461397798,
  'precision': 0.9472089449863148,
  'recall': 0.9472127883827147}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 3.74 , CPU times: user 4min 13s")
print('Final_Result:')
(Final_Result)

RAM: 3.74 , CPU times: user 4min 13s
Final_Result:


[{'accuracy': 95.6278142442538,
  'f1': 0.9562814376054658,
  'precision': 0.9563552970787216,
  'recall': 0.956278142442538}]